# Validate post-2002 commands from PR 250 (ingest pre-2002 commands into v2 archive)

This compares the post-2002 commands generated with PR #250 (Support ingest of
pre-2002 loads) with commands from PR #248 (Major refector of command
interrupts). The commands from PR #248 were validated against flight commands as
of kadi version 6.0.1.

This is essentially a regression test to ensure that the post-2002 command
ingest has not been altered when using the code in PR 250.

In [1]:
import os
from pathlib import Path

import numpy as np

from kadi import __version__, logger
from kadi.commands import commands_v2
from kadi.commands.commands_v2 import clear_caches, get_cmds

In [2]:
print(__version__)

6.0.1


In [3]:
logger.setLevel('DEBUG')

In [4]:
start = "2002:008"
stop = "2022:200"

In [5]:
clear_caches()
os.environ['KADI'] = os.path.abspath('../CMDS_pr248')

In [6]:
cmds_pr248 = get_cmds(start, stop, scenario='flight')
bs_pr248 = cmds_pr248[-300000:].pformat_like_backstop()

2022-09-06 13:53:36,153 get_cmds: Getting commands from archive only
2022-09-06 13:53:36,366 load_idx_cmds: Loaded /Users/aldcroft/git/kadi/CMDS_pr248/cmds2.h5 with 1416689 commands
2022-09-06 13:53:36,797 load_pars_dict: Loaded /Users/aldcroft/git/kadi/CMDS_pr248/cmds2.pkl with 146150 pars


In [9]:
clear_caches()
del commands_v2.PARS_DICT._val
del commands_v2.REV_PARS_DICT._val
os.environ['KADI'] = os.path.abspath('../CMDS_pr250')

In [10]:
cmds_pre2002 = get_cmds(start, stop, scenario='flight')
bs_pre2002 = cmds_pre2002[-300000:].pformat_like_backstop()

2022-09-06 13:55:06,109 get_cmds: Getting commands from archive only
2022-09-06 13:55:06,275 load_idx_cmds: Loaded /Users/aldcroft/git/kadi/utils/cmds2.h5 with 1541474 commands
2022-09-06 13:55:06,635 load_pars_dict: Loaded /Users/aldcroft/git/kadi/utils/cmds2.pkl with 158027 pars


In [11]:
len(cmds_pr248), len(cmds_pre2002)

(1407905, 1407905)

In [12]:
compare_cols = ('date', 'type', 'tlmsid', 'scs', 'step', 'source', 'vcdu')
np.all(cmds_pr248[compare_cols] == cmds_pre2002[compare_cols])

True

In [13]:
bs_pr248 == bs_pre2002

True

In [14]:
bs_pr248[0]

'2018:295:22:44:56.813 | MP_TARGQUAT      | AOUPTARQ   | OCT2118B | cmds=8, q1=-9.25084444e-01, q2=-2.23250400e-01, q3=-2.97779738e-01, q4=7.5533159'